In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import os
os.chdir("../")

from py_trsqp.trsqp import TrustRegionSQPFilter
import numpy as np
import casadi as ca

constants = {}
constants["L_threshold"] = 1.000
constants["eta_1"] = 1E-12
constants["eta_2"] = 1E-11
constants["gamma_0"] = 0.6
constants["gamma_1"] = 0.8
constants["gamma_2"] = 1.5
constants["init_radius"] = 0.5
constants["stopping_radius"] = 1E-16

In [ ]:
def cf(x:np.ndarray) -> np.ndarray: # Rosenbrock function: OF
    return 1*(x[1]-x[0]**2)**2+((x[0]-1)**2)/100

def eq_constraint1(x:np.ndarray) -> np.ndarray: # equality constraints 
    return x[1]**2 + x[0]**2 - 2 # = 0

def eq_constraint2(x:np.ndarray) -> np.ndarray: # equality constraints 
    return x[1] - 1 + - (x[0] - 1)**3 # = 0

def ineq_constraint(x:np.ndarray) -> np.ndarray: # inequality constraints
    return - x[1] - x[0] + 2 # > 0


In [ ]:
tr = TrustRegionSQPFilter(x0 = [-2.5,-2.0],
                          cf = cf, 
                          ub = 6.,
                          lb = -6.,
                          eqcs = [], #[eq_constraint1, eq_constraint2], 
                          ineqcs = [],
                          opts={'solver': 'ipopt', 'budget': 2000}, 
                          constants = constants) #[ineq_constraint])

tr.optimize(max_iter=1000)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import ticker, cm

X, Y = np.meshgrid(np.linspace(-3, 3, 100),
                    np.linspace(-3, 3, 100))
levels = [10**i for i in np.arange(-2,2.5,0.5)]

fsize = 15
tsize = 18
tdir = 'in'
major = 5.0
minor = 3.0

style = 'default'

plt.style.use(style)
plt.rcParams['text.usetex'] = True
plt.rcParams['font.size'] = fsize
plt.rcParams['legend.fontsize'] = tsize
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = major
plt.rcParams['ytick.minor.size'] = minor

# for i in range(len(tr.iterates)):

for i in range(500):
    if i % 1 == 0:
        # print(tr.iterates[i])
        center = tr.iterates[i]['best_point']['y']
        radius = tr.iterates[i]['radius']*np.max([tr.ub[i] - tr.xn[i] for i in range(len(center))])
        intx = X - center[0]
        inty = Y - center[1]
        dist = np.sqrt(intx**2 + inty**2)
        intindices = dist <= radius

        # # func = tr['cf']([X, Y])
        # print(X, Y)
        X_1d = X.flatten(order='F')
        Y_1d = Y.flatten(order='F')
        
        func = cf([X, Y])
        
        # func_map = tr.iterates[i]['models'].m_cf.model.model_polynomial.feval.map(X.shape[0]*X.shape[1])
        func_map = tr.iterates[i]['models'].m_cf.model.model_polynomial_normalized.feval.map(X.shape[0]*X.shape[1])
        x_dm = ca.DM(X).reshape((1, X.shape[0]*X.shape[1]))
        y_dm = ca.DM(Y).reshape((1, X.shape[0]*X.shape[1]))
        func_dm = func_map(ca.vertcat(x_dm, y_dm))
        func_dm = func_dm.reshape((X.shape[0], X.shape[1])).full()
        func[intindices] = func_dm[intindices]

        circle1 = plt.Circle(center, radius, color='black', fill=False)

        fig, ax = plt.subplots(1)
        ax.set_xlabel(f"$x_1$")
        ax.set_ylabel(f"$x_2$")
        ax.add_patch(circle1)
        CS = ax.contour(X, Y, func, levels, norm = LogNorm(), cmap=cm.PuBu_r)
        ax.clabel(CS, fontsize=9, inline=True)
        
        x = []
        y = []
        for p in tr.iterates[i]['models'].m_cf.model.y.T:
            _p = tr.denorm(p)
            x.append(_p[0])
            y.append(_p[1])
        
        plt.scatter(x, y, color='black')
        plt.scatter(x[0], y[0], color='red', label=f'Best point')
        plt.scatter(1.0, 1.0, color='blue', marker='x', label=f'Solution')

        # #Eq constraint    
        # plt.plot(xeq_plot, yeq_plot, 'b-', label=f"eq constraint")

        # #Ineq constraint
        # plt.plot(xineq_plot, yineq_plot, 'r--', label=f"ineq constraint")

        plt.title(f"iteration {i}")
        plt.xlim(-3, 3)
        plt.ylim(-3, 3)
        plt.legend(loc="lower left")
        plt.savefig(f"/home/iffanh/GitRepositories/py-trsqp/examples/results/unconstrained/unconstrained_{i}.png", format="png")
        plt.close()
        
import imageio.v3 as iio

filenames = [f"/home/iffanh/GitRepositories/py-trsqp/examples/results/unconstrained/unconstrained_{i}.png" for i in range(200)]
images = [ ]

for filename in filenames:
    images.append(iio.imread(filename))
    
iio.imwrite(f"/home/iffanh/GitRepositories/py-trsqp/examples/results/unconstrained.gif", images, duration=250, loop=0)

In [ ]:
i

# Constrained

In [ ]:
def ineq1(x):
    return x[0] + x[1]**2

def ineq2(x):
    return x[0]**2 + x[1]

def ineq3(x):
    return x[0]**2 + x[1]**2 - 1

# constants = dict()
# constants["gamma_0"] = 0.7
# constants["gamma_1"] = 0.9
# constants["gamma_2"] = 1.5
# constants["eta_1"] = 1E-8
# constants["eta_2"] = 1E-7
# constants["init_radius"] = 0.2
# constants["stopping_radius"] = 1E-16
# constants["L_threshold"] = 1.0

constants = {}
constants["L_threshold"] = 1.000
constants["eta_1"] = 1E-8
constants["eta_2"] = 0.1
constants["gamma_0"] = 0.6
constants["gamma_1"] = 0.8
constants["gamma_2"] = 1.5
constants["init_radius"] = 1.0
constants["stopping_radius"] = 1E-16
constants["init_radius"] = 0.5

tr = TrustRegionSQPFilter(x0 = [-2.2000, 1.00000],
                          cf = cf, 
                          ub = [0.5, 1.0],
                          lb = [-0.5, -1.0],
                          eqcs = [], #[eq_constraint1, eq_constraint2], 
                          ineqcs = [ineq1, ineq2, ineq3],
                          opts={'solver': 'ipopt', 'budget': 2000}, 
                          constants = constants) #[ineq_constraint])

tr.optimize(max_iter=200)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, colorConverter
from matplotlib import ticker, cm

X, Y = np.meshgrid(np.linspace(-3, 3, 100),
                    np.linspace(-3, 3, 100))
levels = [10**i for i in np.arange(-2,2.5,0.5)]

for i in range(len(tr.iterates)):
    if i % 1 == 0:
        center = tr.iterates[i]['best_point']['y']
        radius = tr.iterates[i]['radius']*np.max([tr.lb[i] - tr.xn[i] for i in range(len(center))])
        intx = X - center[0]
        inty = Y - center[1]
        dist = np.sqrt(intx**2 + inty**2)
        intindices = dist <= radius

        X_1d = X.flatten(order='F')
        Y_1d = Y.flatten(order='F')
        
        func = cf([X, Y])
        
        func_map = tr.iterates[i]['models'].m_cf.model.model_polynomial_normalized.feval.map(X.shape[0]*X.shape[1])
        x_dm = ca.DM(X).reshape((1, X.shape[0]*X.shape[1]))
        y_dm = ca.DM(Y).reshape((1, X.shape[0]*X.shape[1]))
        func_dm = func_map(ca.vertcat(x_dm, y_dm))
        func_dm = func_dm.reshape((X.shape[0], X.shape[1])).full()
        func[intindices] = func_dm[intindices]

        
        circle1 = plt.Circle(center, radius, color='black', fill=False)

        fig, ax = plt.subplots(1)
        ax.set_xlabel(f"$x_1$")
        ax.set_ylabel(f"$x_2$")
        ax.add_patch(circle1)
        CS = ax.contour(X, Y, func, levels, norm = LogNorm(), cmap=cm.PuBu_r)
        ax.clabel(CS, fontsize=9, inline=True)
        
        x = []
        y = []
        for p in tr.iterates[i]['models'].m_cf.model.y.T:
            _p = tr.denorm(p)
            x.append(_p[0])
            y.append(_p[1])
        
        xb = tr.iterates[i]['best_point']['y'][0]
        yb = tr.iterates[i]['best_point']['y'][1]
        
        plt.scatter(x, y, color='black')
        plt.scatter(xb, yb, color='red', label=f'Best point')
        plt.scatter(0.5, 0.5*np.sqrt(3), color='blue', marker='x', label=f'Solution')

        #Ineq constraint
        y = np.linspace(-3,3,100)
        x = - y**2
        # plt.plot(x, y, 'r--')
        ax.fill_between(x, y, [0 for _ in x], alpha=0.2, color='blue')
        
        x = np.linspace(-3,3,100)
        y = - x**2
        # plt.plot(x, y, 'r--')
        ax.fill_betweenx(y, x, [0 for _ in y], alpha=0.2, color='blue')
        
        fc = colorConverter.to_rgba('blue', alpha=0.2)
        circle = plt.Circle((0,0), 1, fill=True, fc=fc)
        ax.add_patch(circle)
        
        
        plt.vlines(0.5, -3, 3, color='green', ls='--')
        plt.vlines(-0.5, -3, 3, color='green', ls='--')

        plt.title(f"iteration {i}")
        plt.xlim(-3, 3)
        plt.ylim(-3, 3)
        plt.legend(loc="upper right")
        plt.savefig(f"/home/iffanh/GitRepositories/py-trsqp/examples/results/constrained/constrained_{i}.png", format="png")
        plt.close()
        
import imageio.v3 as iio

filenames = [f"/home/iffanh/GitRepositories/py-trsqp/examples/results/constrained/constrained_{i}.png" for i in range(len(tr.iterates))]
images = [ ]

for filename in filenames:
    images.append(iio.imread(filename))
    
iio.imwrite(f"/home/iffanh/GitRepositories/py-trsqp/examples/results/constrained.gif", images, duration=250, loop=0)